# **Notebook de test de Téléchargement de donnée de basket depuis le site nba.com**

Sommaire

## Imports

In [2]:
%load_ext autoreload
%autoreload 2
from TelechargementDonnees import JourneeSiteNba, PasDeMatchError, EnsembleJoueursSiteNba, DriverFirefox, gererCookie, simplifierNomJoueur
import pandas as pd
import numpy as np
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException,TimeoutException, ElementClickInterceptedException
from selenium.webdriver.support.ui import Select

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. Telecharger les données liees aux matchs depuis le site internet pour une ou plusieurs dates  
1. on telecharge une journee
1. on sauvegarder dans un json

In [ ]:
#telecharger une journee
try : 
    journee=JourneeSiteNba('2020-12-24', dossierExportCsv=r'C:\Users\martin.schoreisz\git\Basket\Basket\data\testUsa')
    #exporter
journee.saveCsv()
except PasDeMatchError as e : 
    print(e)


In [8]:
# pour un ensemble de journee : 
for j in ['2021-01-%02d' %n  for n in range(1,15)] : 
    print(j)
    try :
        journee=JourneeSiteNba(j,dossierExportCsv=r'C:\Users\martin.schoreisz\git\Basket\Basket\data\testUsa')
        journee.saveCsv()
    except PasDeMatchError as e: 
        print(e)    
        continue

2021-01-01
0 https://www.nba.com/game/mem-vs-cha-0022000069/box-score#box-score
1 https://www.nba.com/game/bos-vs-det-0022000070/box-score#box-score
2 https://www.nba.com/game/mia-vs-dal-0022000071/box-score#box-score
3 https://www.nba.com/game/atl-vs-bkn-0022000072/box-score#box-score
4 https://www.nba.com/game/chi-vs-mil-0022000073/box-score#box-score
5 https://www.nba.com/game/was-vs-min-0022000074/box-score#box-score
6 https://www.nba.com/game/lal-vs-sas-0022000075/box-score#box-score
7 https://www.nba.com/game/phx-vs-den-0022000076/box-score#box-score
8 https://www.nba.com/game/lac-vs-uta-0022000077/box-score#box-score
9 https://www.nba.com/game/por-vs-gsw-0022000078/box-score#box-score
2021-01-02
0 https://www.nba.com/game/sac-vs-hou-0022000079/box-score#box-score
1 https://www.nba.com/game/nyk-vs-ind-0022000080/box-score#box-score
2 https://www.nba.com/game/okc-vs-orl-0022000081/box-score#box-score
3 https://www.nba.com/game/cha-vs-phi-0022000082/box-score#box-score
4 https://ww

## 2. Telecharger les données liees aux joueurs depuis le site internet 
> Dans le cas ou on souhaite telecharger la totalite des donnees de la page des joueuers
1. on se connecte à la page principale des joueurs
1. on affiche l'integralite des donnees

In [62]:
#creation de la classe
pagejoueuer=EnsembleJoueursSiteNba()
#acces aux donnees
pagejoueuer.dfJoueurs
#export des donnees
pagejoueuer.dfJoueurs.reset_index(drop=True).to_json(r'C:\Users\martin.schoreisz\Documents\AffairesEnCours\temp\basket\Joueurs_saison_2020-2021\Joueurs_saison_2020-2021.json')

## 3. Test donnees match sur site americain
1. on se connecte à la page des matchs en fonction de la date
1. on regarde pour telecharger les données
1. on cherche les joueurs
1. on recupere les joueurs inconnus lors de chaque match

In [87]:
driver=DriverFirefox().driver

In [88]:
driver.get('https://www.nba.com/games')
time.sleep(5)

In [89]:
gererCookie(driver)

In [90]:
#par date 
date='2021-01-02'
adresseJournee=f'https://www.nba.com/games?date={date}'
driver.get(adresseJournee)
time.sleep(5)

In [91]:
#recuperer les elements a cliquer
try :
    #containerScore=WebDriverWait(self.driver, 20).until(EC.presence_of_element_located((By.XPATH,f"//div[@class='snapshot-footer']"))) #si besoin que la ligne dessous ne fonctionne pas
    elementsScore=WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.LINK_TEXT, 'BOX SCORE')))  
except TimeoutException :
    raise PasDeMatchError(dateJournee)

In [92]:
#parcourir les elements
for e,p in enumerate([p.get_attribute("href") for p in elementsScore]) : 
    driver.get(p)
    time.sleep(2)
    driver.implicitly_wait(20)
    #sur l'onglet box-score on recupere les stats des equipes
    dfsEquipes=pd.read_html(driver.page_source)
    time.sleep(2)
    driver.implicitly_wait(20)
    #bascule sur l'onglet summary er recuperer les stats de matchs: 
    elementSummary=WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.LINK_TEXT, 'Summary')))
    driver.execute_script("arguments[0].click();", elementSummary)#passer via Javascript pour clicker mm si qqch devant
    time.sleep(2)
    driver.implicitly_wait(20)
    dfMatch=pd.read_html(driver.page_source)[0]

In [83]:
dfMatch

,Unnamed: 0,Q1,Q2,Q3,Q4,FINAL
0,TOR,26,34,22,34,116
1,NOP,32,23,32,33,120


In [84]:
dfsEquipes[0].columns
nomsColonnesStat=['nom', 'minute','tir_reussi','tir_tentes', 'pct_tir', 'trois_pt_r', 'trois_pt_t', 'pct_3_pt','lanc_frc_r', 'lanc_frc_t', 'pct_lfrc',
 'rebonds_o', 'rebonds_d', 'rebonds', 'passes_dec', 'steal','contres','ball_perdu', 'faute_p','points', 'plus_moins']

In [85]:
#mise en forme
df=dfsEquipes[0].copy()
df.drop(df.tail(1).index,inplace=True)
df.columns=nomsColonnesStat
df.loc[df.head(5).index,'nom']=df.head(5).nom.str[:-1]

In [86]:
df

,nom,minute,tir_reussi,tir_tentes,pct_tir,trois_pt_r,trois_pt_t,pct_3_pt,lanc_frc_r,lanc_frc_t,...,rebonds_o,rebonds_d,rebonds,passes_dec,steal,contres,ball_perdu,faute_p,points,plus_moins
0,OG Anunoby,36:26,5,11,45.5,2,8,25.0,5,6,...,1,6,7,3,0,0,2,4,17,2
1,Pascal Siakam,24:44,4,10,40.0,0,4,0.0,2,2,...,2,2,4,1,1,0,3,6,10,-16
2,Aron Baynes,23:02,4,7,57.1,0,1,0.0,0,0,...,1,4,5,1,1,0,0,3,8,-3
3,Kyle Lowry,39:15,4,14,28.6,3,11,27.3,5,5,...,1,5,6,8,0,0,2,3,16,-6
4,Fred VanVleet,39:39,9,23,39.1,4,13,30.8,5,5,...,2,6,8,5,1,1,0,3,27,3
5,Alex Len,7:13,1,2,50.0,0,1,0.0,0,0,...,0,1,1,0,0,2,1,1,2,-5
6,Norman Powell,24:02,4,10,40.0,4,6,66.7,0,0,...,0,2,2,0,0,0,1,2,12,-1
7,Stanley Johnson,11:59,0,1,0.0,0,0,0.0,0,0,...,1,0,1,1,1,0,0,1,0,0
8,Chris Boucher,28:33,7,13,53.8,1,4,25.0,9,9,...,2,3,5,1,2,1,0,5,24,11
9,Terence Davis,5:07,0,0,0.0,0,0,0.0,0,0,...,0,1,1,1,0,0,0,1,0,-5
